# 전처리 & 청킹 테스트

In [1]:
from utils import data_preprocess
preprocessed = data_preprocess("config.json")

/home/hwyewon/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


## config file loaded ##
## raw data (length : 3433) loaded ##


In [2]:
preprocessed[0]

Document(page_content='중도 퇴사 후 근로소득 신고되지 않아 고용허가연장 안된 노동자 지원 작성일: 23-11-27\n상담유형: 체류자격\n거주지역: 여주시\n국적: 우즈베키스탄\n체류자격: E-9\n상담내용: 중도 퇴직하는 외국인 근로자의 추가 납부세액 사례\n진행 과정 및 결과: 2023-09-18\n- 고용노동부에서 외국인근로자의 2020년 종합소득세 체납을   확인하여 고용허가제 연장 불가능 안내받았음 \n 회사 담당자하고 여주세무서에 방문 및 체납된 세금의 원인 상담 내용을 통역 요청으로 전화 상담 \n타행송금; 자동차세-7.360원, 2021년 주민세-11,330원, 2022년 주민세-11,330원, 2022년 지방세 44,630원, 2022년 소득세 – 476,590원, 2021년 지방세 173,140원, \n공과금 2020년 1,808,950원 안내 통역함\n\n2020-09-20\n- 2021년에 연말정산 신청하여 469,520원 환급받았음 \n- 체류자격 연장을 위해서 합계 2,533,330원을 납부함 \n공과금 납부 내용을 상담하러 이천세무서에 동시 통역 가능한지 문의로 전화 상담 \n10월 21일 오전 10시에 이천세무서에 방문 예정 안내함', metadata={'source': '경기도외국인지원센터_상담사례'})

In [ ]:
preprocessed[0].page_content

In [ ]:
import json, os
with open(os.path.join('data/', "merged.json"), 'r') as f:
    data = json.load(f)

titles = []; contents = []

for row in data:
    titles.append(row['title'])
    contents.append(row['content'])

In [ ]:
contents[0]

In [ ]:
import re

# 특수 문자를 제거하고 연속된 공백을 하나로 줄인다.
def remove_escape(raw_text: str) -> str:
    pattern = r"\t|\n|\xa0"
    processed_text = re.sub(pattern, " ", raw_text)
    processed_text_stripped = " ".join(processed_text.split())
    return processed_text_stripped

def remove_hanja(text):
    # Unicode 범위를 사용하여 한자 제거
    return re.sub(r'[\u4e00-\u9fff]+', '', text)

#하이퍼링크 제거
def remove_hyperlink(raw_text: str) -> str:
    pattern = (
        r":*\s*\(*:*\s*https?://[\w\dㄱ-ㅎㅏ-ㅣ가-힣!@#$%^&*(),.?/:;\"'<>{}|+=~_-]+\s*\)*"
    )
    processed_text = re.sub(pattern, "", raw_text)
    return processed_text

#텍스트 시작 부분 헤더 제거
def remove_header(raw_text: str) -> str:
    header_pattern = "안녕하십니까. 대한법률구조공단 사이버상담을 이용해 주셔서 감사합니다."
    header_end_idx = re.search(header_pattern, raw_text)
    if header_end_idx != None:
        processed_text = raw_text[header_end_idx.end() :]
        return processed_text
    else:
        return raw_text

#텍스트 끝 부분 푸터 제거
def remove_footer(raw_text: str) -> str:
    footer_pattern = "※ 주의 : 사례에 대한 답변은 법령이나 판례 등의 변경으로 내용이 바뀔 수 있으므로 구체적인 사안에 대해서는 반드시 대한법률구조공단 상담(전화상담은 국번없이 ☎ 132) 등을 통해 다시 한 번 확인하시기 바랍니다."
    footer_start_idx = re.search(footer_pattern, raw_text)
    if footer_start_idx != None:
        processed_text = raw_text[: footer_start_idx.start()]
        return processed_text
    else:
        return raw_text

def remove_author_and_url(text):
    # 작성자 정보 제거
    text = re.sub(r'작성자:\s*[\w\s]+', '', text)

    # URL 제거
    text = re.sub(r'URL:\s*https?://\S+', '', text)

    # 마지막 줄바꿈 제거
    text = text.strip()

    return text

#특정 키워드가 포함된 문장 제거
def remove_page_word(raw_text: str) -> str:
    pattern = '사이버상담|사이버 상담|공단|방문|국번없이 132|132번'
    if re.findall(pattern, raw_text) == []:
        return raw_text

    split_text = raw_text.split('.')
    remove_text = [i for i in split_text if re.findall(pattern, i) == []]

    return '.'.join(remove_text)

def remove_phone_number(raw_text: str) -> str:
    pattern = r'\b(\d{2,3}-\d{3,4}-\d{4}|\d{2}-\d{3}-\d{4})\b'
    processed_text = re.sub(pattern, "", raw_text)
    return processed_text

In [ ]:
tmp = [contents[0], contents[1]]

In [ ]:
preprocess_functions = [
    remove_hanja,
    remove_header,
    remove_footer,
    #remove_escape,
    remove_phone_number,
    #remove_page_word,
    remove_hyperlink,
    remove_author_and_url,
    #remove_link,
]

for preprocess_function in preprocess_functions:
    tmp = list(map(preprocess_function, tmp))


In [ ]:
len(tmp[1])

In [ ]:
tmp[1]

In [ ]:
len(contents[1])

## template 확인

In [9]:
with open('templates/chat_template.txt', 'r') as f:
    template = f.readlines()

In [10]:
''.join(template)

'당신은 유능한 AI 어시스턴트 (Assistant) 입니다. [관련 문서]를 참조하여 [질문]에 대한 적절한 [답변]을 생성해주세요.\n\n[관련 문서]\n{context}\n\n[질문]\n{question}\n\n[답변]\n'